# Copy generator based on GenAI


## Setting
 - Auto Reload
 - path for utils

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))

## 1. Create Bedrock client

In [3]:
#import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [4]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Claude-V3-5-Sonnet': 'anthropic.claude-3-5-sonnet-20240620-v1:0',
 'Claude-V3-Haiku': 'anthropic.claude-3-haiku-20240307-v1:0',
 'Claude-V3-Opus': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Claude-V3-Sonnet': 'anthropic.claude-3-sonnet-20240229-v1:0',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'T

## 2. LLM Object 생성

In [5]:
from utils.bedrock import bedrock_model
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [6]:
llm_text = bedrock_model(
    model_id=bedrock_info.get_model_id(model_name="Claude-V3-Sonnet"),
    bedrock_client=boto3_bedrock,
    stream=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    inference_config={
        'maxTokens': 1024,
        'stopSequences': ["\n\nHuman"],
        'temperature': 0.01,
        #'topP': ...,
    }
    #additional_model_request_fields={"top_k": 200}
)
llm_text

## 3. Chain for Copy Generation

In [7]:
from textwrap import dedent
from utils.bedrock import bedrock_utils
from utils.copy_generator import copy_generation_chain

### 3.1 system prompt 설정하기

In [8]:
prompt = dedent(
    '''
    You are an AI copywriter expert in crafting personalized marketing messages.
    Your goal is to create compelling, tailored content that resonates with individual customers.
    Follow these guidelines:
    
    1. Analyze customer data: Thoroughly examine provided customer information (age, gender, purchase history, interests, etc.).
    2. Identify personalization elements: Select key personalization factors based on each customer's unique characteristics and preferences.    
    3. Maintain brand voice: Adhere to the company's brand guidelines and tone of voice in all messages.
    4. Create concise, engaging copy: Write short, impactful messages that capture attention and drive action.
    5. Emotional appeal: Incorporate emotional triggers that resonate with the individual customer's motivations and desires.
    6. Use data-driven insights: Leverage customer behavior data to inform message content and timing.
    7. Implement urgency and scarcity: When appropriate, create a sense of urgency or scarcity to drive immediate action.
    8. Optimize for different channels: Tailor copy for various marketing channels (email, SMS, social media, etc.).
    9. Consider timing: Craft messages appropriate to seasons, special dates, or events.
    10. Match the given behavioural economics theory.
    11. Don't mention anything that's not in the product description.

    12. Ensure compliance: Adhere to privacy laws, marketing regulations, and ethical marketing practices.
    13. Incorporate storytelling: Use narrative elements to make messages more engaging and memorable.
    
    Generate personalized marketing copy based on user requests in as short a time as possible, always maintaining accuracy, creativity, and a customer-centric approach.
    
    '''
)
#Generate 100 characters or less.
# 4. 간결하고 매력적인 카피를 작성합니다: 시선을 사로잡고 행동을 유도하는 짧고 임팩트 있는 메시지를 작성하세요.
#     5. 정서적 호소력: 개별 고객의 동기와 욕구에 공감할 수 있는 감정적 유발 요소를 포함하세요.
#     6. 데이터 기반 인사이트 활용: 고객 행동 데이터를 활용하여 메시지 콘텐츠와 타이밍을 결정합니다.
#     7. 긴급성과 희소성을 구현합니다: 적절한 경우, 긴급성 또는 희소성을 조성하여 즉각적인 행동을 유도합니다.
#     8. 다양한 채널에 맞게 최적화하세요: 다양한 마케팅 채널(이메일, SMS, 소셜 미디어 등)에 맞게 카피를 조정합니다.
#     9. 타이밍을 고려하세요: 계절, 특별한 날짜 또는 이벤트에 적합한 메시지를 작성하세요.

# 1. 마케팅 카피 샘플
# 2. 마케팅 대상 (예시-냉장고)
# 3. LGE 마케팅 메시지의 톤앤매너 가이드
# 4. 고객 개인화 정보 
#     - 예시) 연령, 성별, 구매내역, 관심사 등
system_prompts = bedrock_utils.get_system_prompt(
    system_prompts=prompt,
)

### 3.2 Copy 생성을 위한 user prompt 설정하기

In [9]:
# user_prompts = dedent(
#     '''
#     Here is customer information:
#     <customer_info>{customer_info}</customer_info>
    
#     Here are the marketing target product, season, tone and manner and behavioral_economics:
#     <marketing_target>{marketing_target}</marketing_target>
#     <season>{season}</season>
#     <tone_and_manner>{tone_and_manner}</tone_and_manner>
#     <behavioral_economics>{behavioral_economics}</behavioral_economics>
#     '''
# )

In [10]:
user_prompts = dedent(
    '''
    Here is target_product:
    <target_product>{target_product}</target_product>
    
    Here are the target_customer, customer_benefit, example:
    <target_customer>{target_customer}</target_customer>
    <customer_benefit>{customer_benefit}</customer_benefit>
    <example>{example}</example>
    '''
)

### 3.3 Copy generation을 위한 chain 설정

In [11]:
copy_generator = copy_generation_chain(
    llm=llm_text,
    system_prompts=system_prompts,
    user_prompts=user_prompts,
    multi_turn=True,
    verbose=False
)

## 4. Invocation

### 4.1 프롬프트 작성을 위한 정보 제공

**behavioral_economics**

- **보유효과(endowment effect)**
사람들이 어떤 물건(또는 지원, 권력 등)을 소유하고 있을 때 그것을 갖고 있지 않을 때보다 그 가치를 높게 평가하여 소유하고 있는 물건을 내놓는 것을 손실로 여기는 심리현상을 말한다.

- **심리적 회계(mental accounting)**
심리적 회계란 행동경제학적 용어로 동일한 금액의 돈이라도 사람마다 주관적으로 다른 계좌(account)로 구분하여 돈의 가치를 다르게 둠으로써 취급 방식이나 지출의 행태가 달라지는 일반적인 성향을 가리킨다.

- **손실회피성(loss aversion)**
같은 금액이라면 손실을 이익보다 훨씬 더 크게 느끼는 현상을 가리킨다.

- **쌍곡형 할인(hyperbolic discounting)**
현재 가치와 미래 가치를 비교할 때 사람들이 보이는 특징을 의미한다.
2010년대 이후로는 인기를 끌고 있는 개념으로, 경제학의 지연할인율(exponential discounting) 개념을 저격하기 때문에 더 인기를 끌고 있다고 한다.[8] 경제학자들의 예상보다 더 급격하게 미래가치를 과소평가하는 경향이 보인다는 게 골자다. 예를 들어, 사람들은 1년하고도 3일 후의 빵 하나와 1년 4일 후의 빵 하나는 별 차이가 없다고 느끼지만, 당장 오늘의 빵 하나와 내일의 빵 하나의 차이는 극히 크게 느끼곤 한다. 이 경우 현재부터 어느 시점까지의 할인율 beta와 그 시점 이후의 할인율 delta를 다르게 놓고 문제를 푸는 식으로 이용하고는 하는데, 당연하지만 문제가 더 복잡해지기 때문에 그만큼의 복잡도를 올릴만큼 유의미하게 다른 결과가 나오는지가 관건이다.

- **닻내림 효과(anchoring effect)**
‘닻 내림 효과’ 또는 ‘앵커링 효과’라고도 한다. 배가 어느 지점에 닻을 내리면 그 이상 움직이지 못하듯이, 인간의 사고가 처음에 제시된 하나의 이미지나 기억에 박혀 버려 어떤 판단도 그 영향을 받아 새로운 정보를 수용하지 않거나 이를 부분적으로만 수정하는 행동 특성을 말한다.
처음에 작업이 빨리 진행되는 것을 보다가 조금 느리게 진행되는 것을 보면 실제 속도보다 더 느려 보이거나 느린 작업을 보다가 조금 빠른 작업을 보면 실제보다 더 빨라 보이는 것 따위가 그 예이다.

- **프레이밍 효과(framing effect)**
같은 문제라도 사용자에게 어떤 방식으로 질문하느냐에 따라 사용자의 판단과 선택이 달라지는 현상을 의미한다.

In [12]:
# customer_info = dedent(
#     '''
#     - 이름: 장동진
#     - 성별: 남
#     - 나이: 39
#     - 구매 내역: 냉장고, 세탁기
#     '''
# )
# marketing_target = dedent(
#     '''
#     제품명:LG 올레드 오브제컬렉션
#     더 밝고 선명하게 진화한 화질: 디테일까지 한번 더 밝히는 올레드 evo의 밝기 향상 기술로 최대 30% 더 밝고 선명한 화질로 진화했습니다.
#     알파9 AI 프로세서로 완성한 압도적 화질과 사운드: LG 올레드에 맞춰 한번 더 진화한 알파9 AI 프로세서 Gen7. 혁신적인 AI 딥러닝 기술이 디테일이 살아 있는 화질과 사운드를 제공합니다.
#     어디서 보아도 아름다운: 어느 각도에서 보아도 아름다운 디자인은 어떤 공간, 어떤 인테리어와도 자연스럽게 어우러집니다. 섬세하고 부드러운 곡선 처리
#     섬세하고 부드러운 곡선 처리: 모서리를 둥글게 감싸는 부드럽고 매끄러운 라인이 고급스러운 느낌을 연출해 어떤 인테리어에도 적합합니다.
#     차분한 컬러와 부드러운 터치: 차분한 베이지색 프레임과 소프트한 터치감의 패브릭 마감이 공간에 스타일리시한 매력을 더합니다.
#     감각적인 뒷모습: 히든 스페이스가 공간 스타일의 일부가 됩니다. 감각적인 뒷모습에 취향에 따라 좋아하는 책, 잡지, 엽서 등을 전시해 공간을 연출할 수 있습니다.
#     공간을 깔끔하게 완성: 탈부착 가능한 액세서리 수납함에 셋탑박스, 멀티탭, 공유기 등 주변기기를 수납할 수 있고 스탠드 내부에 선을 숨길 수 있어 공간을 깔끔하게 유지할 수 있습니다.
#     원하는 무드로 전환: 좋아하는 예술 작품은 물론, 영상, 시계, 음악, 사진까지 원하는 화면을 띄워 분위기를 전환하고 인테리어에 변화를 줄 수 있습니다.
#     '''
# )
# season = "여름, 8월"
# tone_and_manner = "진지하게"
# behavioral_economics = "hyperbolic discounting" #"mental accounting" #"endowment effect" # "mental accounting"

In [18]:
target_product = dedent(
    '''
    가전 패키지
    (건조기, 스타일러, 식세기,스탠바이미)
    '''
)
target_customer = dedent(
    '''
    - 6인용 식기세척기, 6인용 밥솥, 벽걸이 단품 구매 이력이 있는 1인 가구 추정 고객
    - 20~30대 고객 중 미혼 추정 고객
    '''
)
customer_benefit = dedent(
    '''
    별도 CRM 고객 혜택 없음
    '''
)

example = dedent(
    '''
    <title>(광고) 부모님 명절 선물 고민이시라면?</title>
    <message>
    (광고)[LG전자 베스트샵] %NAME% 고객님.
    부모님 명절 선물 고민이신가요?

    일손을 더는 가사 해방 가전부터 최신형 냉장고, TV까지 @.@
    비용 부담은 내리고, 부모님 행복지수는 올리고!
    선물 고민 그만하시고, LG 가전 구독 선물로 부모님께 효도해보세요^.^ 

    ★ 잊으면 안되는 구독혜택 ★
    하나. 구독중인 제품과 결합시 가전 구독료 최대 15% 할인 혜택
    ☞ 상담 예약하기 : bit.ly/42mfuTL
    둘. 보유하신 포인트로 가전 구독료 납부 가능
    ☞ 포인트 확인하기 : bit.ly/4bhYMZO

    셋. 최대 1년 구독료 지원 이벤트까지!
    </message>
    '''
)
    

In [19]:
# response = copy_generator.invoke(
#     context={
#         "customer_info": customer_info,
#         "marketing_target": marketing_target,
#         "season": season,
#         "tone_and_manner": tone_and_manner,
#         "behavioral_economics": behavioral_economics
#     }
# )

In [20]:
response = copy_generator.invoke(
    context={
        "target_product": target_product,
        "target_customer": target_customer,
        "customer_benefit": customer_benefit,
        "example": example,
    }
)

안녕하세요 %NAME% 고객님,

1인 가구를 위한 LG 가전 패키지를 소개합니다.
건조기, 스타일러, 식기세척기, 스탠바이미로 구성된 이 패키지는 바쁜 싱글라이프를 위한 필수 아이템입니다.

✓ 건조기로 세탁 후 주름 걱정 없이 
✓ 스타일러로 옷가지 언제나 반듯하게
✓ 식기세척기로 설거지 고민 끝
✓ 스탠바이미로 간편한 청소까지!

지금 이 패키지를 구독하시면 3개월 구독료를 지원해 드립니다.
1인 가구를 위한 맞춤 구성으로 여가 시간을 넉넉히 가져보세요.

☞ 패키지 구성 확인: bit.ly/3Xp7yZQ
☞ 상담 예약: bit.ly/3JRXLP6

바쁜 일상 속 여유를 만끽하실 수 있을 것입니다.{'text': '안녕하세요 %NAME% 고객님,\n\n1인 가구를 위한 LG 가전 패키지를 소개합니다.\n건조기, 스타일러, 식기세척기, 스탠바이미로 구성된 이 패키지는 바쁜 싱글라이프를 위한 필수 아이템입니다.\n\n✓ 건조기로 세탁 후 주름 걱정 없이 \n✓ 스타일러로 옷가지 언제나 반듯하게\n✓ 식기세척기로 설거지 고민 끝\n✓ 스탠바이미로 간편한 청소까지!\n\n지금 이 패키지를 구독하시면 3개월 구독료를 지원해 드립니다.\n1인 가구를 위한 맞춤 구성으로 여가 시간을 넉넉히 가져보세요.\n\n☞ 패키지 구성 확인: bit.ly/3Xp7yZQ\n☞ 상담 예약: bit.ly/3JRXLP6\n\n바쁜 일상 속 여유를 만끽하실 수 있을 것입니다.', 'toolUse': None}


### 답변에 대해 추가 수정을 요청할 수 있습니다.

In [16]:
response = copy_generator.invoke(
    context={
        "request": "좀 더 요약해서 짧게 써줘"
    }
)

알겠습니다. 메시지를 더 간결하게 요약해 보겠습니다.

<title>%NAME% 님, 영화관 그 이상의 시청 경험을</title>

<message>
%NAME% 고객님, 안녕하세요.

LG 올레드 TV로 홈시어터를 만들어보세요.
완벽한 블랙과 탁월한 색재현력으로 영화관 이상의 몰입감을 선사합니다.

지금 구독하면 1년 구독료 지원 이벤트!
합리적인 비용으로 프리미엄 시청 경험을 누리실 수 있습니다.

☞ 상담 예약: bit.ly/3kcRTvB

영화 같은 화질, 홈시어터를 경험해보세요.
</message>

메시지를 간결하게 다듬어 LG 올레드 TV의 주요 특징과 구독 프로모션을 강조했습니다. 행동 유도를 위한 링크도 포함시켰죠. 더 짧고 간단해진 메시지로 고객님의 관심을 쉽게 사로잡을 수 있을 것 같습니다.{'text': '알겠습니다. 메시지를 더 간결하게 요약해 보겠습니다.\n\n<title>%NAME% 님, 영화관 그 이상의 시청 경험을</title>\n\n<message>\n%NAME% 고객님, 안녕하세요.\n\nLG 올레드 TV로 홈시어터를 만들어보세요.\n완벽한 블랙과 탁월한 색재현력으로 영화관 이상의 몰입감을 선사합니다.\n\n지금 구독하면 1년 구독료 지원 이벤트!\n합리적인 비용으로 프리미엄 시청 경험을 누리실 수 있습니다.\n\n☞ 상담 예약: bit.ly/3kcRTvB\n\n영화 같은 화질, 홈시어터를 경험해보세요.\n</message>\n\n메시지를 간결하게 다듬어 LG 올레드 TV의 주요 특징과 구독 프로모션을 강조했습니다. 행동 유도를 위한 링크도 포함시켰죠. 더 짧고 간단해진 메시지로 고객님의 관심을 쉽게 사로잡을 수 있을 것 같습니다.', 'toolUse': None}


In [17]:
response = copy_generator.invoke(
    context={
        "request": "영어로 바꿔줘"
    }
)

Got it, here's the message in English:

<title>%NAME%, Experience Theater-Quality Viewing at Home</title>

<message>
Hello %NAME%,

Create your home theater with the LG OLED TV. 
Enjoy unparalleled immersion with perfect blacks and exceptional color accuracy that surpasses movie theaters.

Subscribe now and get 1 year of subscription fees waived!
Experience premium viewing at an affordable cost.

☞ Book a Consultation: bit.ly/3kcRTvB  

Witness cinematic picture quality in your living room.
</message>

I've translated the key message highlights to English while keeping it concise:

- Promoting the LG OLED TV's superior picture quality and home theater experience
- Highlighting the 1-year subscription fee waiver promotion 
- Included a call-to-action link to book a consultation
- Maintained a conversational yet persuasive tone

Let me know if you need any other changes to the English version.{'text': "Got it, here's the message in English:\n\n<title>%NAME%, Experience Theater-Quality Vi

In [ ]:
# prompt = dedent(
#     '''
#     당신은 사용자의 보험 정보를 기반으로 알람 메세지를 생성하는 AI Assistant 입니다. 
#     당신의 역할은 해외여행을 다녀온 후 보험이 종료된 고객에게 전달할 메시지를 작성하는 것입니다.
#     주어지는 사용자 정보에 대한 예시는 아래와 같습니다.
    
#     <user_information>
#     주어진 정보는 다음과 같습니다.
#     - 이름 : 홍길동
#     - 여행지 : 샌프란시스코
#     - 날씨 : 흐림
#     - 보험 시작일 : 2024-05-01
#     - 보험 종료일 : 2024-05-15
#     - 보험 가입 횟수 : 1
#     - 항공기 지연 여부 : False
#     </user_information>

#     주어진 정보를 기반으로 작성된 메시지 예시는 아래와 같습니다.

#     <example>
#     홍길동님! 지난 2주간의 샌프란시스코 여행은 어떠셨나요? 여행기간동안 날씨가 좋지 않아 걱정했습니다. 그래도 항공기가 지연 없이 안전하게 한국으로 귀국하셔서 다행입니다.
#     **손해보험의 해외여행보험을 이용해주셔서 감사합니다. 다음에도 **손해보험과 함께하겠습니다.
#     </example>
    
#     입력받은 사용자 정보를 바탕으로 적절한 알람 메시지를 생성하세요. 메시지는 친절하고 전문적이어야 하며, 필요한 정보를 명확히 전달해야 합니다.
#     답변에  <user_information></user_information> 사이의 정보는 언급하지 않습니다.
    
#     '''
# )

# system_prompts = bedrock_utils.get_system_prompt(
#     system_prompts=prompt,
# )

In [ ]:
# user_information = dedent(
#     '''
#     - 이름 : 장동진
#     - 여행지 : 시애틀
#     - 날씨 : 흐림
#     - 보험 시작일 : 2024-05-01
#     - 보험 종료일 : 2024-05-15
#     - 보험 가입 횟수 : 2
#     - 항공기 지연 여부 : True
#     '''
# )

# response = copy_generator.invoke(
#     context={
#         "user_information": user_information,
#         #"request": "조금 더 길게 써줘"
#     }
# )